#Imports

In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Dataset Preprocessing

In [142]:
data = pd.read_csv("/content/Test Data - Sheet1.csv")
df = data[["item", "exercise", "clothes", "electronics"]]
df.head(10)

,item,exercise,clothes,electronics
0,GOYOUTH 2 in 1 Under Desk Electric Treadmill M...,1,0,0
1,Adjustable Dumbbells 55LB Single Dumbbell 5 in...,1,0,0
2,GOFITNESS Push Down Bar Machine - Chest Expand...,1,0,0
3,"Cubii JR2, Under Desk Elliptical, Under Desk B...",1,0,0
4,Resistance Bands for Working Out- 5 Pack Loop ...,1,0,0
5,"Niceday Steppers for Exercise, Stair Stepper w...",1,0,0
6,"RENRANRING Resistance Bands for Working Out, 1...",1,0,0
7,Adjustable Weight Bench - Utility Weight Bench...,1,0,0
8,FitBeast Hand Grip Strengthener Workout Kit (5...,1,0,0
9,Gaiam Essentials Thick Yoga Mat Fitness & Exer...,1,0,0


In [143]:
class DataPreprocessing:
    def __init__(self, data):
        self.data = data

    @staticmethod
    def str_cols(data):
        str_cols = []
        for col in data.columns:
            if isinstance(data[col][0], str):
                str_cols.append(col)
        return str_cols

    @staticmethod
    def lower_case(string):
        return string.lower()

    @staticmethod
    def letters_only(word):
        letters = set('abcdefghijklmnopqrstuvwxyz ')
        word = ''.join(char for char in word if char in letters)
        return word

    @staticmethod
    def space_offset(string):
        offset = 0
        for char in string:
            if char == ' ':
                offset += 1
            else:
                return offset

    @staticmethod
    def remove_mult_spaces(s):
        spaces = False
        i = 0
        while i < len(s):
            if s[i] == ' ' and spaces:
                offset = DataPreprocessing.space_offset(s[i:])
                s = s[:i] + s[i+1+offset:]
            elif s[i] == ' ' and not spaces:
                spaces = True
            else:
                spaces = False
            i += 1
        return s

    def text_preprocess(self, col):
        for i, itemName in enumerate(self.data[col]):
            itemName = self.lower_case(itemName)
            itemName = self.letters_only(itemName)
            itemName = self.remove_mult_spaces(itemName)
            self.data.loc[i, col] = itemName
        return None

data_preprocessing = DataPreprocessing(df)
data_preprocessing.text_preprocess("item")


In [144]:
df['item']

0      goyouth nnder desk electric treadmill motorize...
1      adjustable dumbbells lb single dumbbell nree d...
2      gofitness push down bar machine hest expander ...
3      cubii jr under desk elliptical under desk bike...
4      resistance bands for working out ack loop exer...
                             ...                        
145    high pressure shower head owerful deluxe bathr...
146    beam electronics car phone holder mount phone ...
147     in harging station for apple devices mag afe ...
148    telescopes for adults mm aperture and mm focal...
149    walkers razor quad electronic muffs ic egree s...
Name: item, Length: 150, dtype: object

# Analysis

In [113]:
df[df["exercise"] == 1]

,item,exercise,clothes,electronics
0,goyouth nnder desk electric treadmill motorize...,1,0,0
1,adjustable dumbbells lb single dumbbell nree d...,1,0,0
2,gofitness push down bar machine hest expander ...,1,0,0
3,cubii jr under desk elliptical under desk bike...,1,0,0
4,resistance bands for working out ack loop exer...,1,0,0
5,niceday steppers for exercise stair stepper wi...,1,0,0
6,renranring resistance bands for working out lb...,1,0,0
7,adjustable weight bench tility weight benches ...,1,0,0
8,fitbeast hand grip strengthener workout kit ac...,1,0,0
9,gaiam essentials thick yoga mat fitness xercis...,1,0,0


In [145]:
def popCount(col):
    popularity_dict = {}
    word_arr = []
    for item in col:
      words = item.split(' ')
      for word in words:
        if word not in popularity_dict:
          popularity_dict[word] = 1
          word_arr.append(word)
        else:
          popularity_dict[word] += 1
    return popularity_dict, word_arr


def popWords(word_dict, word_arr):
    wordPopArr = []
    for word in word_arr:
      wordPopArr.append([word, word_dict[word]])
    return wordPopArr



def sortPopWords(popWordArr):
    for i in range(len(popWordArr)):
      for j in range(i+1, len(popWordArr)):
        if popWordArr[i][-1] < popWordArr[j][-1]:
          popWordArr[i], popWordArr[j] = popWordArr[j], popWordArr[i]
    return popWordArr



def returnPopWords(data):
  word_dict, word_Arr = popCount(data["item"])
  popWordArr = popWords(word_dict, word_Arr)
  sortedPopWordArr = sortPopWords(popWordArr)
  return sortedPopWordArr

cols = df.columns
data_dicts = []

for i in range(1, len(cols)):
    filtered_data = df[df[cols[i]] == 1]
    data_dicts.append(returnPopWords(filtered_data))

print(data_dicts[2])



[['for', 41], ['with', 32], ['charger', 19], ['iphone', 15], ['and', 15], ['wireless', 14], ['charging', 13], ['lock', 12], ['portable', 11], ['door', 10], ['fast', 9], ['projector', 9], ['air', 9], ['phone', 9], ['compatible', 9], ['apple', 7], ['bluetooth', 7], ['headphones', 7], ['max', 7], ['display', 6], ['kids', 6], ['light', 6], ['battery', 6], ['pro', 6], ['usb', 6], ['electric', 5], ['led', 5], ['electronic', 5], ['airpods', 5], ['outdoor', 5], ['p', 5], ['smart', 5], ['w', 5], ['mini', 5], ['power', 5], ['station', 5], ['drum', 5], ['k', 4], ['', 4], ['hd', 4], ['in', 4], ['tv', 4], ['case', 4], ['c', 4], ['cleaning', 4], ['earphones', 4], ['pd', 4], ['laser', 4], ['sound', 4], ['high', 4], ['amazon', 3], ['echo', 3], ['fire', 3], ['mfi', 3], ['white', 3], ['ear', 3], ['buds', 3], ['magnetic', 3], ['dash', 3], ['front', 3], ['mah', 3], ['camera', 3], ['cable', 3], ['night', 3], ['adults', 3], ['games', 3], ['lectronic', 3], ['certified', 3], ['life', 3], ['movie', 3], ['nirel

In [147]:
exercise_dict = {
    "exercise": True,
    "bands": True,
    "resistance": True,
    "workout": True,
    "fitness": True,
    "equipment": True,
    "gym": True,
    "bike": True,
    "yoga": True,
    "training": True,
    "mat": True,
    "adjustable": True,
    "exerciser": True,
    "physical": True,
    "pilates": True,
    "elliptical": True,
    "lbs": True,
    "stretching": True,
    "weight": True,
    "strength": True
}

clothes_dict = {
    "mens": True,
    "womens": True,
    "sleeve": True,
    "long": True,
    "workout": True,
    "shirts": True,
    "tops": True,
    "pants": True,
    "causal": True,
    "fleece": True,
    "ribbed": True,
    "shirt": True,
    "yoga": True,
    "sweatshirt": True,
    "pullover": True,
    "women": True,
    "slim": True,
    "piece": True,
    "jacket": True,
    "button": True,
    "clothes": True,
    "fit": True,
    "zip": True,
    "shorts": True,
    "seamless": True,
    "jogger": True,
    "hoodie": True
}

electronics_dict = {
    "charger": True,
    "wireless": True,
    "iphone": True,
    "fast": True,
    "bluetooth": True,
    "apple": True,
    "display": True,
    "electronic": True,
    "headphones": True,
    "smart": True,
    "usb": True,
    "led": True,
    "battery": True,
    "electric": True,
    "airpods": True,
    "laser": True,
    "tv": True,
    "sound": True,
    "hd": True,
    "earphones": True,
    "tablet": True,
    "echo": True,
    "amazon": True
}

title_arr = []

for item in df["item"]:
    input_arr = []
    words = item.split(" ")
    for word in words:
        if word in exercise_dict or word in clothes_dict or word in electronics_dict:
            input_arr.append(word)
    title_arr.append(input_arr)

print(title_arr)


[['electric', 'exercise', 'wireless', 'led', 'display'], ['adjustable', 'gym'], ['workout', 'equipment', 'exerciser', 'resistance', 'exercise', 'gym'], ['elliptical', 'bike', 'exerciser', 'elliptical', 'fitness', 'elliptical', 'exerciser', 'exercise'], ['resistance', 'bands', 'exercise', 'bands', 'resistance', 'workout', 'bands', 'stretching', 'fitness', 'bands', 'yoga', 'pilates'], ['exercise', 'resistance', 'bands', 'lbs', 'fitness'], ['resistance', 'bands', 'lbs', 'exercise', 'bands', 'workout', 'bands', 'resistance', 'bands', 'women', 'training', 'physical'], ['adjustable', 'weight', 'weight', 'workout', 'exercise', 'gym'], ['workout', 'adjustable', 'resistance', 'exerciser'], ['yoga', 'mat', 'fitness', 'mat', 'yoga', 'mat'], ['elliptical', 'bike', 'exerciser', 'elliptical', 'fitness', 'elliptical', 'exerciser', 'exercise'], ['exercise', 'bike', 'resistance', 'display', 'lbs', 'bike', 'bike', 'gym', 'workout'], ['women', 'adjustable', 'workout', 'fitness', 'exercise'], ['gym', 'str

In [155]:
df["item"] = title_arr

<ipython-input-155-2f49539232fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["item"] = title_arr


In [156]:
df

,item,exercise,clothes,electronics
0,"[electric, exercise, wireless, led, display]",1,0,0
1,"[adjustable, gym]",1,0,0
2,"[workout, equipment, exerciser, resistance, ex...",1,0,0
3,"[elliptical, bike, exerciser, elliptical, fitn...",1,0,0
4,"[resistance, bands, exercise, bands, resistanc...",1,0,0
...,...,...,...,...
145,[fit],0,0,1
146,[iphone],0,0,1
147,"[apple, charger, fast, charger, wireless, iphone]",0,0,1
148,[],0,0,1


#Data Split

In [157]:
def data_split(data):
    data = data.sample(frac=1)
    features = data["item"].tolist()
    labels = []

    for i in range(len(data)):
        if data["exercise"][i] == 1:
            labels.append("exercise")
        elif data["clothes"][i] == 1:
            labels.append("clothes")
        else:
            labels.append("electronics")

    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=42
    )

    return train_features, train_labels, test_features, test_labels

trainData, trainLabels, testData, testLabels = data_split(df)


In [166]:
def maxSeqLen(arr):
  max_len = 0
  for arr in title_arr:
    if len(arr) > max_len:
      max_len = len(arr)
  return max_len

def vocabSize(arr):
  occurArr = []
  for seq in arr:
    for word in seq:
      if word not in occurArr:
        occurArr.append(word)
  return len(occurArr)

max_seq_len = maxSeqLen(title_arr)
vocab_size = vocabSize(title_arr)
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
label_encoder = LabelEncoder()

all_labels = trainLabels + testLabels
label_encoder.fit(all_labels)

train_labels_encoded = label_encoder.transform(trainLabels)
test_labels_encoded = label_encoder.transform(testLabels)

def model(data, labels):
    tokenizer.fit_on_texts(data)
    trainSeq = tokenizer.texts_to_sequences(data)
    trainPadded = pad_sequences(trainSeq, maxlen=max_seq_len, padding='post', truncating='post')
    train_data_padded = trainPadded

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_seq_len),
        tf.keras.layers.LSTM(units=64, return_sequences=False),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

    model.fit(train_data_padded, labels, epochs=50, validation_split=0.2, batch_size=32)

    return model


trained_model = model(trainData, train_labels_encoded)


model(trainData, train_labels_encoded)

Epoch 1/50
3/3 [==============================] - 3s 332ms/step - loss: 1.1017 - accuracy: 0.2917 - val_loss: 1.1013 - val_accuracy: 0.2917
Epoch 2/50
3/3 [==============================] - 0s 27ms/step - loss: 1.0966 - accuracy: 0.3542 - val_loss: 1.1094 - val_accuracy: 0.2917
Epoch 3/50
3/3 [==============================] - 0s 25ms/step - loss: 1.0945 - accuracy: 0.3854 - val_loss: 1.1183 - val_accuracy: 0.1667
Epoch 4/50
3/3 [==============================] - 0s 43ms/step - loss: 1.0923 - accuracy: 0.4062 - val_loss: 1.1263 - val_accuracy: 0.2083
Epoch 5/50
3/3 [==============================] - 0s 25ms/step - loss: 1.0910 - accuracy: 0.4062 - val_loss: 1.1347 - val_accuracy: 0.2083
Epoch 6/50
3/3 [==============================] - 0s 34ms/step - loss: 1.0882 - accuracy: 0.3958 - val_loss: 1.1393 - val_accuracy: 0.2083
Epoch 7/50
3/3 [==============================] - 0s 25ms/step - loss: 1.0863 - accuracy: 0.3958 - val_loss: 1.1475 - val_accuracy: 0.2083
Epoch 8/50
3/3 [==========

https://www.amazon.com/BCAN-Foldable-Trampoline-Rebounder-Adjustable/dp/B07SMGQ18F/ref=sr_1_52_sspa?crid=2DC2LR788AF2L&keywords=exercise&qid=1697913520&sprefix=exercise%2Caps%2C152&sr=8-52-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA&th=1&psc=1